In [1]:
import random
from numpy.random import choice
import numpy as np
import matplotlib.pyplot as plt
 

#Bounds [-10,10]

#generate populations
   
    
# the initialization function to generate initial random populations
def initialization():
    population=[]
    for i in range(5):
        population.append([])
        for j in range(5):
            #population[i].append(random.randint(-10,10))
            population[i].append(random.uniform(-10,10))
    
    print("initial population :",population)
    return population
 
#calculate fitness value for bounded trial vectors(input:2D list)
def calcFitness(population,func):
    score = []
    
    for i in range(5):
        #print("fitness:",func(population[i]))
        if func == "Bent":
            score.append(bent_cigar_func(population[i]))
        elif func == "Discus":
            score.append(discus_func(population[i]))
        elif func == "HCE":
            score.append( HCE_func(population[i]))
        elif func == "Rosenbrock":
            score.append(rosenbrock_func(population[i]))
    return score
                     
#  Bent cigar function (input:1D list)
def bent_cigar_func(trail_vector):
    result = pow(trail_vector[0],2)
   
    for i in range(1,5):    
        result+=pow(10,6)*pow(trail_vector[i],2)
    return result

# Discus function
def discus_func(trail_vector):
    result = pow(trail_vector[0],2)*pow(10,2)
    for i in range(1,5):
        result+=pow(trail_vector[i],2)
    return result

# High Conditioned Elliptic function
def HCE_func(trail_vector):
    result = 0
    for i in range(5):
        result+=pow(10,6*(i/4))*pow(trail_vector[i],2)
    return result

# Rosenbrock's function
def rosenbrock_func(trail_vector):
    result=0
    for i in range(4):
        result+=100*pow( pow(trail_vector[i],2)-trail_vector[i+1], 2 )+pow(trail_vector[i]-1,2)
    return result

def mutation(population, targetIndex,F=0.8):
    donor_vector=[]
    #randomly choose from the population matrix
    picklist=[0,1,2,3,4]
    picklist.remove(targetIndex) #to prevent select same index as the target vector, we remove the target vector index
    #print(picklist)
    
    temp=np.random.choice(picklist,3,replace=False)
    first_vector = population[temp[0]]
    #print("V1:",first_vector)
    
    second_vector = population[temp[1]]
    #print("V2:",second_vector)
    
    third_vector = population[temp[2]]
    #print("V3:",third_vector)
    for i in range(5):
        donor_vector.append(first_vector[i]+F*(second_vector[i]-third_vector[i]))
    return donor_vector

# Binomial crossover to determine the trail_vector
def crossover(donor_vector,target_vector,Cr=0.9):
    trial_vector=[]
    r_random=[]
    for j in range(5):
        r_trial=random.choice([0,1])
        r_random.append(r_trial)
        
    location = random.choice([0,1,2,3,4])
    
    for k in range(len(donor_vector)):
        if r_random[k] <= Cr or k == location:
            trial_vector.append(donor_vector[k])
        elif r_random[k] > Cr and k != location:
            trial_vector.append(target_vector[k])
  
    return trial_vector
 
# Adjust bounds of offspring
def bound(x):
    for i in range(len(x)):
        if x[i] <= 10 and x[i] >= -10:
            pass
        elif x[i] > 10:
            x[i]= 10
        elif x[i] < -10:
            x[i]= -10
    return x

# Greedy select population function
def select(next_g,population,current_score,next_score):
    solution=[]
    
    for i in range(5):
        if next_score[i] <= current_score[i]:
            solution.append(next_g[i])
         
        else:
            solution.append(population[i])
          
    return solution


    
# In[2]:
#######main process Here#############
def sovler(func_name):
# NP=100 number of population at first generation
# D=5    dimension
#Cr=0.9  Crossover factor
#F=0.8   Mutation factor

#MAXFES= 5000*D the maximum number of generations
    NP=100
    Cr=0.9
    F=0.8
    MAXFES=25000
    donor_vector = []
    target_vector = []
    trail_vector = []
    bounded_vector = []
    population = []
    population = initialization()
    next_score = []
    solution=[]
   
    
    # Calculate current fitness score for this population
    for i in range(MAXFES):
    #print("Round:",i)
        
        
        # Generation phase
        
        for j in range(NP):
            current_score = calcFitness(population,func_name) #[45000049, 10000004, 110000009, 115000016, 115000049]
        
        #print("Current_score :",current_score)
        #print("current_population",population)
            next_g=[]
            m=0
            while m<5 :
                target_vector= population[m]
                 # Mutate the population
                donor_vector=mutation(population,m)
                # Compare each element in both vector and determine the survive individual for the next generation
                trail_vector=crossover(donor_vector,target_vector)
                # Control the value of X to ensure its within [-10,10]
                bounded_vector=bound(trail_vector)
                
                next_g.append(bounded_vector) #bounded population U
            #print("target_v:",target_vector)
            #print("donor_v:",donor_vector)
            #print("trail_v:",trail_vector)
               
               
                m+=1
          # Get a fitness score for the modified population
        #print("Next_g",next_g) 
            next_score = calcFitness(next_g,func_name) #[159000100, 114000016, 198000081, 187000081, 98000025]    
        # Selection phase
     
        # Perform greedy selection for next generation and update population and fitness
        
        #next_g=select(trail_vector,target_vector,current_score,next_score)
            population=select(next_g,population,current_score,next_score)
        #print("new_population",population)
        
    minvalue = min(next_score)
    solution = population[next_score.index(minvalue)]
    return minvalue,solution
    #print("F(x)min:",minvalue)
    #print("Solution:",population[next_score.index(minvalue)])
solution_1=[]
min_1,solution_1= sovler("Bent")
print("Minimum of function:",min_1)
print("Solution:",solution_1)

solution_2=[]
#min_2,solution_2= sovler("Discus")
#print("Minimum of function:",min_2)
#print("Solution:",solution_2)

solution_3=[]
#min_3,solution_3= sovler("HCE")
#print("Minimum of function:",min_3)
#print("Solution:",solution_3)

solution_4=[]
#min_4,solution_4= sovler("Rosenbrock")
#print("Minimum of function:",min_4)
#print("Solution:",solution_4)


initial population : [[8.408045549374435, -4.43938556492637, -3.7481860015737256, 5.482469234397339, 2.3879355221371323], [1.0457482688415318, -8.282510960716296, -1.882308304525358, 4.013186251341569, -0.1409594626677979], [0.9314565771923462, 7.481705038718186, -0.3800308273953057, -3.275955130394486, 2.4299074737322854], [-8.774367601422728, -8.38718807435477, 9.97262810304878, 3.2767732783703547, -8.859798030675762], [7.312717012885599, -1.6946501816658568, -2.4898502226242165, -7.119849120352082, -6.536815980387118]]
Minimum of function: 5.6633578957517064e-05
Solution: [0.0075255284836028, 5.362393441142973e-14, -4.4349703157191174e-14, 5.339505517767741e-14, -3.9603058000529144e-14]
